# Resume Screening
-  https://www.analyticsvidhya.com/blog/2021/06/resume-screening-with-natural-language-processing-in-python/
-  https://github.com/anukalp-mishra/Resume-Screening/blob/main/Resume_Screening.ipynb
-  https://github.com/Oindrila-Sen/Python-Projects/blob/master/Resume_Scanner/resume_scanner.ipynb
-  https://towardsdatascience.com/pdfs-to-word-cloud-in-3-steps-73ccbff6d835
Libraries
-  PyPDF2 : to parse the PDF file (method 1)
-  textract : to parse the PDF file (method 1)
-  wordcloud : to create the word cloud image
-  nltk : to get the stopwords and to tokenize the string to keywords
-  collection : to get the mapping of keywords to its occurrence count

In [25]:
#install
#!pip install docx2txt
#!pip install PyPDF2
#!pip install pdfreader
#!pip install pdfminer
#!pip install textract

In [26]:
import numpy as np
import pandas as pd
import docx2txt
import os

In [27]:
import io
import os
import PyPDF2
import textract
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

# Docx resume
import docx2txt

#Wordcloud
import re
import operator
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
set(stopwords.words('english'))
from wordcloud import WordCloud
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
#pdf scanner
def read_pdf_resume(pdf_doc):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(pdf_doc, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            
        text = fake_file_handle.getvalue()
    
    # close open handles
    converter.close()
    fake_file_handle.close()
    
    if text:
        return text

In [30]:
# word resume
def read_word_resume(word_doc):
     resume = docx2txt.process(word_doc)
     resume = str(resume)
     #print(resume)
     text =  ''.join(resume)
     text = text.replace("\n", "")
     if text:
         return text

In [31]:
# JD from file
def read_JD_from_word(word_doc):
     jdText = docx2txt.process(word_doc)
     jdText = str(jdText)
     #print(resume)
     text =  ''.join(jdText)
     text = text.replace("\n", "")
     if text:
        return text

In [32]:
#clean JD
def clean_job_description(jd):
     ''' a function to create a word cloud based on the input text parameter'''
     ## Clean the Text
     # Lower
     clean_jd = jd.lower()
     # remove punctuation
     clean_jd = re.sub(r'[^\w\s]', '', clean_jd)
     # remove trailing spaces
     clean_jd = clean_jd.strip()
     # remove numbers
     clean_jd = re.sub('[0-9]+', '', clean_jd)
     # tokenize 
     clean_jd = word_tokenize(clean_jd)
     # remove stop words
     stop = stopwords.words('english')
     clean_jd = [w for w in clean_jd if not w in stop] 
     return(clean_jd)

In [33]:
#Create a word cloud:
# create_word_cloud(jd, wd=50): max_words = wd, 
def create_word_cloud(jd):
    corpus = jd
    fdist = FreqDist(corpus)
    #print(fdist.most_common(100))
    words = ' '.join(corpus)
    words = words.split()

    # create a empty dictionary
    data = dict()
    #  Get frequency for each words where word is the key and the count is the value
    for word in (words):
        word = word.lower()
        data[word] = data.get(word, 0) + 1
    # Sort the dictionary in reverse order to print first the most used terms    
    dict(sorted(data.items(), key=operator.itemgetter(1),reverse=True))
    word_cloud = WordCloud(width = 800, height = 800, background_color ='white',max_words = 500)
    word_cloud.generate_from_frequencies(data)
    # plot the WordCloud image                        
    plt.figure(figsize = (10, 8), edgecolor = 'k') 
    plt.imshow(word_cloud,interpolation = 'bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.show()

In [34]:
#Get Job Description and Resume Match Score
def get_resume_score(cvName, text):
    cv = CountVectorizer(stop_words='english')
    count_matrix = cv.fit_transform(text)
    #Print the similarity scores
    print("\nSimilarity Scores:")
     
    #get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal
     
    print("Your resume " + str(cvName) + " matches about " + str(matchPercentage)+ "% of the job description.")

In [35]:
def read_file_pypdf(filepath):
    pdfFileObj = open(filepath,'rb')
    pdfReader = PyPDF2.pdfReader(pdfFileObj)
    num_pages = reader.pages(pg)
    text = ""
    # Read all the pages
    for pg in range(num_pages):
        page = pdfReader.getPage(pg)
        text += page.extractText()
    return text 

In [36]:
def read_file_textract(filepath, use_method = 'textract'):
    
    text = ""
    if not os.path.isfile(filepath):
        print(f'Invalid file:{filepath}')
    else:
        if use_method == 'textract':
            return read_file_textract(filepath)
        elif use_method == 'pypdf':
            return read_file_pypdf(filepath)
        else:
            print('Invalid method to read file. Supported formats: "textract" or "pypdf".')
    
    return text

In [37]:
def extract_keywords(text, ignore_words = [],   min_word_length = 0, ignore_numbers = True,  ignore_case = True):
    # Remove words with special characters
    filtered_text = ''.join(filter(lambda x:x in string.printable, text))
    
    # Create word tokens from the text string
    tokens = word_tokenize(filtered_text)
    
    # List of punctuations to be ignored 
    punctuations = ['(',')',';',':','[',']',',','.','--','-','#','!','*','"','%']
    
    # Get the stopwords list to be ignored
    stop_words = stopwords.words('english')

    # Convert ignore words from user to lower case
    ignore_words_lower = [x.lower() for x in ignore_words]
    
    # Combine all the words to be ignored
    all_ignored_words = punctuations + stop_words + ignore_words_lower
    
    # Get the keywords list
    keywords = [word for word in tokens \
                    if  word.lower() not in all_ignored_words
                    and len(word) >= min_word_length]    

    # Remove keywords with only digits
    if ignore_numbers:
        keywords = [keyword for keyword in keywords if not keyword.isdigit()]

    # Return all keywords in lower case if case is not of significance
    if ignore_case:
        keywords = [keyword.lower() for keyword in keywords]
    
    return keywords

In [38]:
def create_word_cloud2(keywords, maximum_words = 100, bg = 'white', cmap='Dark2', maximum_font_size = 256, width = 3000, height = 2000,
                     random_state = 42, fig_w = 15, fig_h = 10, output_filepath = None):
    
    # Convert keywords to dictionary with values and its occurences
    word_could_dict=Counter(keywords)

    wordcloud = WordCloud(background_color=bg, max_words=maximum_words, colormap=cmap, 
                          stopwords=STOPWORDS, max_font_size=maximum_font_size,
                          random_state=random_state, 
                          width=width, height=height).generate_from_frequencies(word_could_dict)
    
    plt.figure(figsize=(fig_w,fig_h))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    if output_filepath:
        plt.savefig(output_filepath, bbox_inches='tight')
    plt.show()
    plt.close()

In [44]:
#name='main'
jdFile = 'E:/auData/hr/cv/cvset1/jdFaculty.docx'
cvPDF = 'E:/auData/hr/cv/cvset1/DrJAnitha.pdf'
cvName ='DrJAnitha'
#cvWord  = 'E:/auData/hr/cv/cvset1/DrSwaranlatha.docx'
', '.join(os.listdir('E:/auData/hr/cv/cvset1/'))
filepath=cvPDF
os.listdir(filepath)

NotADirectoryError: [WinError 267] The directory name is invalid: 'E:/auData/hr/cv/cvset1/DrJAnitha.pdf'

In [40]:
#manual way
filepath=cvPDF
file_text = read_file_pypdf(filepath)
keywords = extract_keywords(file_text, min_word_length = 3)
create_word_cloud2(keywords,maximum_words=200)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/auData/hr/cv/DrJAnitha.pdf'

In [ ]:
#Test Resume Scanner:
if __name__ == '__main__':
    extn = input("Enter File Extension: ")
    #print(extn)
    if extn == "pdf":
        resume = read_pdf_resume(cvPDF)
    else:
        resume = read_word_resume(cvWord)
    
    #JD  = input("\nEnter the Job Description: ") 
    JD =  read_JD_from_word(jdFile) 
    ## Get a Keywords Cloud 
    clean_jd = clean_job_description(JD) 
    print('Word Cloud of KeyWords of Job Desciption')
    create_word_cloud(clean_jd)
    
    print("Word Cloud of Keywords from CV")
    create_word_cloud(resume)
    create_word_cloud(clean_job_description(cvWord))
    text = [resume, JD] 
    
    ## Get a Match score
    get_resume_score(cvName, text)

Job Description: University Faculty Member
Position Overview:
The University Faculty Member is a dedicated and passionate educator responsible for delivering high-quality instruction, conducting research, and contributing to the academic and professional growth of students within a specific discipline. This role involves engaging in scholarly activities, collaborating with colleagues, and participating in curriculum development to maintain and enhance the university's academic standards.
Key Responsibilities: 
1. Teaching and Instruction: 
   - Design and deliver engaging and effective course materials that align with the curriculum and learning objectives.
   - Conduct lectures, seminars, workshops, and practical sessions to facilitate student understanding and skill development.
   - Provide timely and constructive feedback on assignments, projects, and examinations to support student progress.
2. Research and Scholarship:
   - Conduct original research in the relevant field, contributing to the advancement of knowledge and the discipline.
   - Publish research findings in peer-reviewed journals, present at conferences, and participate in scholarly discussions.
   - Seek and secure research grants and funding to support research endeavours.
3. Student Mentorship:
   - Advise and mentor students on academic, research, and career-related matters.
   - Foster a supportive and inclusive learning environment that promotes student success and personal growth.
   - Assist students in developing critical thinking, problem-solving skills, and a strong academic foundation.

4. Curriculum Development:
   - Collaborate with colleagues to develop and update course curricula, ensuring relevance to industry trends and academic standards.
   - Integrate innovative teaching methods, technology, and experiential learning opportunities to enhance the learning experience.

5. Service and Collaboration:
   - Participate in departmental meetings, committees, and academic governance activities to contribute to the university's decision-making processes.
   - Collaborate with colleagues to develop interdisciplinary programs, research initiatives, and community engagement projects.
   - Engage with professional organizations and external stakeholders to foster partnerships and promote the university's reputation.

6. Professional Development:
   - Stay current with developments in the field through continuous learning, attending conferences, workshops, and other professional development opportunities.
   - Pursue tenure and promotion milestones as per university guidelines, demonstrating excellence in teaching, research, and service.

Qualifications:

- A doctoral degree (Ph.D. or equivalent) in the relevant field.
- Strong passion for teaching, demonstrated by previous teaching experience, evaluations, and/or awards.
- Active research agenda with a record of scholarly publications and presentations.
- Excellent communication and interpersonal skills to effectively engage with students, colleagues, and external partners.
- Commitment to diversity, equity, and inclusion in teaching, research, and service activities.
- Ability to work collaboratively in a team-oriented academic environment.
- Proficiency in utilizing technology and innovative teaching methods for enhancing student learning.
- Experience in curriculum development and assessment is a plus.
- Previous grant-writing and research funding acquisition experience is desirable.
